# Insert Mall Lat-Long Data

This sheet aims to collect all shopping malls data in Singapore, together with their long and lat for data processing

In [30]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

import re

In [31]:
df = pd.read_excel("shopping_malls.xlsx", header=None)

In [32]:
df

,0
0,100 AM
1,313@Somerset
2,Aperia
3,Balestier Hill Shopping Centre
4,Bugis Cube
...,...
161,Fairprice Hub
162,Gek Poh Shopping Centre
163,Rochester Mall
164,Taman Jurong Shopping Centre


In [33]:
import json
import requests
import urllib.parse
import time

# For each row, get the long and lat coordinates
for i in range(df.shape[0]):
    address = df[0][i]
    query_address = urllib.parse.quote(address)
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y&pageNum=1'
    resp = requests.get(query_string)

    #Convert JSON into Python Object 
    data=json.loads(resp.content)

    if(data['found'] == 0): 
        print("no data in row:", i)
    else:
        df.loc[i, 'Longitude'] = data['results'][-1]['LONGITUDE']
        df.loc[i, 'Latitude'] = data['results'][-1]['LATITUDE']
        
    if((i+1)%250==0):
        print("Sleeping...", i)
        time.sleep(15)
        

no data in row: 9
no data in row: 12
no data in row: 19
no data in row: 30
no data in row: 40
no data in row: 43
no data in row: 158


In [34]:
# Will fill in these data manually in the exported csv as there are only a few of them
df[df['Latitude'].isnull()]

,0,Longitude,Latitude
9,Clarke Quay Central,NaN,NaN
12,City Gate Mall,NaN,NaN
19,Holland Village Shopping Mall,NaN,NaN
30,Mustafa Shopping Centre,NaN,NaN
40,PoMo,NaN,NaN
43,Shaw House and Centre,NaN,NaN
158,OD Mall,NaN,NaN


In [35]:
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
df = df.rename(columns={0:'Mall'})

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Mall       166 non-null    object 
 1   Longitude  159 non-null    float64
 2   Latitude   159 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.0+ KB


#### Just for fun, I wanted to see where the mallsare located in Singapore

In [37]:
# Plot points on a map to get a general sensing of how malls are distributed in Singapore
import plotly.express as px
map_fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', hover_name='Mall')
map_fig.show()

In [38]:
df.to_csv('malls_data.csv', index=False)